## Document for the problem

### The Warehouse Location Problem

We use the warehouse location problem throughout this chapter, which considers
the optimal locations to build warehouses to meet delivery demands. Let `N` be a
set of candidate warehouse locations, and let `M` be a set of customer locations. For
each warehouse `n`, the cost of delivering product to customer `m` is given by $ d_{n,m} $.
We wish to determine the optimal warehouse locations that will minimize the total
cost of product delivery. The binary variables $y_n$ are used to define whether or not a
warehouse should be built, where $y_n$ is 1 if warehouse `n` is selected and 0 otherwise.
The variable $x_{n,m}$ indicates the fraction of demand for customer m that is served by
warehouse `n`.


The variables `x` and `y` are determined by the optimizer, and all other quantities are
known inputs or parameters in the problem. This problem is a particular description
of the p-median problem, and it has the interesting property that the `x` variables will
converge to {0, 1} even though they are not specified as binary variables.
The complete problem formulation is:

Objective:
$$
min \sum_{n \in N}{\sum_{m \in M}{d_{n,m}x_{n,m}}}
$$

s.t.:
$$
\sum_{n \in N}{x_{n,m}} = 1, \forall m \in M
$$

$$
x_{n,m} \leq y_n, \forall n \in N, m \in M
$$

$$
\sum_{n \in N}{y_n} \leq P
$$

$$
0 \leq x \leq 1
$$

$$
y \in {0, 1}
$$

Here, the objective is to minimize the total cost associated with delivering products to all the customers. Equation WL.2 ensures that each customer’s
demand is fully met, and equation WL.3 ensures that a warehouse can deliver product to customers only if that warehouse is selected to be built. Finally, with equation WL.4 the number of warehouses that can be built is limited to `P`.
For our example, we will assume that $P = 2$, with the following data for warehouse and customer locations,

For our example, we will assume that $P = 2$, with the following data for warehouse and customer locations,
```
Customer locations            M = {‘NYC’, ‘LA’, ‘Chicago’, ‘Houston’}
Candidate warehouse locations N = {‘Harlingen’, ‘Memphis’, ‘Ashland’}
```
with the costs $d_{n,m}$ as given in the following table:


|           | NYC  | LA   | Chicago | Houston |
| --------- | ---- | ---- | ------- | ------- |
| Harlingen | 1956 | 1606 | 1410    | 330     |
| Memphis   | 1096 | 1792 | 531     | 567     |
| Ashland   | 485  | 2322 | 324     | 1236    |

In [ ]:
import os
import pandas as pd
import pyomo.environ as pyo

# import ipymaterialui as mui
import ipywidgets as widgets
from IPython.display import display

# import ipyvuetify as v

## Read the data from csv using pandas

---

In [ ]:
data_select = widgets.RadioButtons(
    options=['Upload', 'Select'],
    value='Select'
)

In [ ]:
data_source = widgets.Dropdown(description='Data file:')
uploader = widgets.FileUpload(
        accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
        multiple=False  # True to accept multiple files upload else False
)

In [ ]:
def create_data_file_dd(data_source):
    data_file_list = list()

    cwd = os.getcwd()
    for item in os.listdir(cwd):
        if os.path.isfile(os.path.join(cwd, item)) and item.lower().endswith(".csv"):
            data_file_list.append(item)

    default_value = data_file_list[0] if data_file_list else None
    
    data_source.options = data_file_list
    data_source.value = default_value

    display(data_source)

In [ ]:
out = widgets.Output()

@out.capture(clear_output=True, wait=True)
def display_data_source(*args):
    if data_select.value == "Select":
        create_data_file_dd(data_source)
    else:
        display(uploader)


In [ ]:
display_data_source()
data_select.observe(display_data_source, 'value')

In [ ]:
display(widgets.HBox([data_select, out]))

In [ ]:
import io

def load_data(csv_file_path) -> pd.DataFrame:
    if data_select.value == 'Upload':
        df = pd.read_csv(io.BytesIO(uploader.get_state()['data'][0]), index_col=0)
    else:
        df = pd.read_csv(csv_file_path, index_col=0)
    return df

### Show the data


In [ ]:
sd_button = widgets.Button(description='Show the data')
out_data = widgets.Output()

display(sd_button, out_data)

data = None

def on_button_click(b):
    global data
    with out_data:
        data = load_data(data_source.value)
        print(data)

sd_button.on_click(on_button_click)

In [ ]:

def create_model(df: pd.DataFrame) -> pyo.ConcreteModel:
    N = list(df.index.map(str))
    M = list(df.columns.map(str))
    d = {(r, c):df.loc[r,c] for r in N for c in M}
    P = 2

    model = pyo.ConcreteModel(name = "(WL)")
    
    model.N = pyo.Set(initialize=N)
    model.M = pyo.Set(initialize=M)
    model.x = pyo.Var(N, M, bounds=(0, 1))
    model.y = pyo.Var(N, within=pyo.Binary)

    def obj_rule(model):
        return sum(d[n,m]*model.x[n,m] for n in N for m in M)

    model.obj = pyo.Objective(rule=obj_rule)

    def one_per_cust_rule(model, m):
        return sum(model.x[n,m] for n in N) == 1

    model.one_per_cust = pyo.Constraint(M, rule=one_per_cust_rule)

    def warehouse_active_rule(model, n, m):
        return model.x[n,m] <= model.y[n]

    model.warehouse_active = pyo.Constraint(N, M, rule=warehouse_active_rule)

    def num_warehouses_rule(model):
        return sum(model.y[n] for n in N) <= P
    model.num_warehouses = pyo.Constraint(rule=num_warehouses_rule)
    return model

## Create the model

---

In [ ]:
model = None

cm_button = widgets.Button(description='Create the model')
out_model = widgets.Output()

display(cm_button, out_model)

def on_m_button_click(b):
    global model
    global data
    
    if data is None:
        data = load_data(data_source.value)
        
    with out_model:
        model = create_model(data)
        model.pprint()

cm_button.on_click(on_m_button_click)

## Solve the model

---

In [ ]:
sm_button = widgets.Button(description='Solve the model')
out_res = widgets.Output()

display(sm_button, out_res)

def on_sm_button_click(b):
    global model
    global data
    
    if data is None:
        data = load_data(data_source.value)
        
    with out_res:
        solver = pyo.SolverFactory('glpk')
        res = solver.solve(model)
        
        print(res)
        
        # post process
        # produce nicely formatted output
        for wl in model.N:
            if pyo.value(model.y[wl]) > 0.5:
                customers = [str(cl) for cl in model.M if pyo.value(model.x[wl, cl] > 0.5)]
                print(f"{wl} + serves customers: {customers}")
            else:
                print(str(wl)+": do not build")

sm_button.on_click(on_sm_button_click)